In [22]:
# import for preprocessing
import pandas as pd
import duckdb
from rich import print
import logging
import sys
import sqlite3

# llama index import to create sql DB
from llama_index import (
    SQLDatabase,
    SimpleDirectoryReader,
    Document,
    StorageContext,
    ServiceContext,
    VectorStoreIndex,
    OpenAIEmbedding,
    set_global_service_context,
)
from llama_index.indices.struct_store import (
    NLSQLTableQueryEngine,
    SQLTableRetrieverQueryEngine,
)
from llama_index.llms import OpenAI

# import for text 2 SQl query engine
from llama_index.indices.struct_store.sql_query import (
    NLSQLTableQueryEngine,
    SQLTableRetrieverQueryEngine,
)

# iport if we do not know the name of table ahead of time
from llama_index.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)

# import to set up query tool
from llama_index.tools.query_engine import QueryEngineTool

# import to set up query router
from llama_index.query_engine.router_query_engine import RouterQueryEngine
from llama_index.selectors.llm_selectors import LLMSingleSelector

# import to set up weaviate
from llama_index.vector_stores import WeaviateVectorStore

import weaviate
import openai

from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())
import os
openai.api_key = os.getenv("OPENA_AI_KEY")

from IPython.display import Markdown, display


from sqlalchemy import engine, MetaData, create_engine, text, select

# Import modules for self correcting
from llama_index.query_engine import RetryQueryEngine
from llama_index.evaluation import RelevancyEvaluator
from llama_index.query_engine import RetrySourceQueryEngine

# Import modules to customize guidelines for what response shou look like
from llama_index.evaluation.guideline import (
    GuidelineEvaluator,
    DEFAULT_GUIDELINES,
)
from llama_index.response.schema import Response
from llama_index.indices.query.query_transform.feedback_transform import (
    FeedbackQueryTransformation,
)
from llama_index.query_engine.retry_query_engine import (
    RetryGuidelineQueryEngine,
)

# import to display llama index response
from llama_index.response.notebook_utils import display_response
# Needed for running async functions in Jupyter Notebook
import nest_asyncio

nest_asyncio.apply()

# Tru Lens Evaluation
# import sys
# sys.path.append(os.path.dirname(find_dotenv()))
# from utils import get_prebuilt_trulens_recorder
# from trulens_eval import Tru

# Import for senetnce window
from utils import build_sentence_window_index
from utils import  build_sentence_window_index_vector_DB, get_sentence_window_query_engine
from utils import (build_automerging_index_vector_DB, 
build_automerging_index, get_automerging_query_engine)


In [2]:
auth_config = weaviate.AuthApiKey(api_key="c9la4QfNWbdKafDMbIevGgpvsqRwbu8vuESZ")

client = weaviate.Client(
  url="https://product-automerging-rerank-btez10df.weaviate.network",
  auth_client_secret=auth_config
)

# 1. INGESTION

## Load the product data from file

In [3]:
filepath = "/teamspace/studios/Data_Studio/product_listing/product_listing.csv"
data = pd.read_csv(filepath)
data.columns = ['Product_Name', 'Price', 'Rating', 'Description', 'Features']
print(data.columns)

Index(['Product_Name', 'Price', 'Rating', 'Description', 'Features'], dtype='object')

## Create an sqlite DB for the product data

In [5]:
# Create a connection to an SQLite database file
conn = sqlite3.connect('./data/product_sqlite.db')

# Write the DataFrame to a table in the SQLite database
data.to_sql('product_table', conn, if_exists='replace', index=False)

12

In [6]:
# View the sqlite DB

result = conn.execute("SELECT * FROM product_table limit 1").fetchall()

# Process the result
for row in result:
    rprint(row)

(
    'Ultimate Wireless Bluetooth Earbuds',
    79.99,
    4.4,
    "Elevate your music experience with our Ultimate Wireless Bluetooth Earbuds. These earbuds are designed for 
audiophiles who demand premium sound quality and convenience. With advanced Bluetooth 5.0 technology, you'll enjoy 
seamless connectivity and crystal-clear audio.",
    '- High-fidelity sound with deep bass and clear treble. - Noise-canceling technology for an immersive listening
experience. - 20 hours of playtime on a single charge. - Ergonomic design for a secure and comfortable fit. - Touch
controls for easy playback and calls. - IPX7 waterproof rating for workouts and outdoor activities. - Built-in 
microphone for hands-free calls.'
)

## Create SQLDatabase Object

In [4]:
# SQLITE
engine = create_engine("sqlite:///" +"./data/product_sqlite.db")
connection = engine.connect()
result = connection.execute(text("SELECT * FROM product_table limit 1"))
for row in result:
    print(row)


('Ultimate Wireless Bluetooth Earbuds', 79.99, 4.4, "Elevate your music experience with our Ultimate Wireless 
Bluetooth Earbuds. These earbuds are designed for audiophiles who demand premium sound quality and convenience. 
With advanced Bluetooth 5.0 technology, you'll enjoy seamless connectivity and crystal-clear audio.", '- 
High-fidelity sound with deep bass and clear treble. - Noise-canceling technology for an immersive listening 
experience. - 20 hours of playtime on  ... (72 characters truncated) ... Touch controls for easy playback and 
calls. - IPX7 waterproof rating for workouts and outdoor activities. - Built-in microphone for hands-free calls.')

## Define SQL Database

In [5]:
llm = OpenAI(temperature=0.1, model = "gpt-3.5-turbo")
embed_model = OpenAIEmbedding(model="text-embedding-3-small")
# service_context = ServiceContext.from_defaults(llm =llm, embed_model=embed_model,)
# set_global_service_context(service_context)
# service_context = ServiceContext.from_defaults(llm =llm)
sql_database = SQLDatabase(engine, include_tables=["product_table"])

## Text to SQL Query Engine

In [6]:
query_engine = NLSQLTableQueryEngine(
    sql_database = sql_database,
    tables = ["product_table"])
query_str = "Which product was sold for 79.99?"
response = query_engine.query(query_str)
display_response(response)

**`Final Response:`** The products that were sold for $79.99 are the Ultimate Wireless Bluetooth Earbuds and the Portable Solar Charger.

## Part 2: Query-Time Retrieval of Tables for Text-to-SQL
If we don’t know ahead of time which table we would like to use, and the total size of the table schema overflows your context window size, we should store the table schema in an index so that during query time we can retrieve the right schema.

The way we can do this is using the SQLTableNodeMapping object, which takes in a SQLDatabase and produces a Node object for each SQLTableSchema object passed into the ObjectIndex constructor.

In [7]:

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="product_table"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

In [8]:
query_str = "What is the Rating for the product whose name is Portable Solar Charger?"
response = query_engine.query(query_str)
display(Markdown(f"<b>{response}</b>"))

<b>The rating for the product named Portable Solar Charger is 4.0.</b>

In [11]:
# you can also fetch the raw result from SQLAlchemy!
response.metadata["result"]

[(4.0,)]

## Create embedding for the columns to embed

In [9]:
## With PANDAS DATAFRAME
# Assuming you have already loaded the data into a Pandas DataFrame df
columns_to_embed = ["Description", "Features"]
columns_to_metadata = ["Product Name", "Price", "Rating", "Description", "Features"]

docs = []
for i, row in data.iterrows():
    to_metadata = {col: row[col] for col in columns_to_metadata if col in row}
    values_to_embed = {k: row[k] for k in columns_to_embed if k in row}
    to_embed = '\n'.join(f"{k.strip()}: {v.strip()}" for k, v in values_to_embed.items())
    newDoc = Document(text=to_embed, metadata=to_metadata)
    docs.append(newDoc)

# Create a single document from a list of Documents
# this is what we will chunk up and store with its embedding in Weaviate
document = Document(text="\n\n".join([doc.text for doc in docs]))

## Create Weaviate Schema

In [11]:
# Create Schema
product_schema = {
   "classes": [
       {
           "class": "Product",
           "description": "Listing of Products",
           "vectorizer": "text2vec-openai",
           "properties": [
               {
                  "name": "Product_Name",
                  "dataType": ["text"],
                  "description": "Name of the product.",
               },
               {
                  "name": "Price",
                  "dataType": ["number"],
                  "description": "Price of the product.",
               },
               {
                  "name": "Rating",
                  "dataType": ["number"],
                  "description": "Rating of the product.",
               },
               {
                  "name": "Description",
                  "dataType": ["text"],
                  "description": "Description of the product.",
               },
               {
                  "name": "Features",
                  "dataType": ["text"],
                  "description": "Feature of the product.",
               }
            ]
        }
    ]
}

client.schema.create(product_schema)
print("Product schema was created.")

Product schema was created.

In [10]:
# view the schme just created
print(client.schema.get())

{
    'classes': [
        {
            'class': 'Product',
            'description': 'Listing of Products',
            'invertedIndexConfig': {
                'bm25': {'b': 0.75, 'k1': 1.2},
                'cleanupIntervalSeconds': 60,
                'stopwords': {'additions': None, 'preset': 'en', 'removals': None}
            },
            'moduleConfig': {
                'text2vec-openai': {
                    'baseURL': 'https://api.openai.com',
                    'model': 'ada',
                    'modelVersion': '002',
                    'type': 'text',
                    'vectorizeClassName': True
                }
            },
            'multiTenancyConfig': {'enabled': False},
            'properties': [
                {
                    'dataType': ['text'],
                    'description': 'Name of the product.',
                    'indexFilterable': True,
                    'indexSearchable': True,
                    'moduleConfig': {'text2vec-openai': {'skip': False, 'vectorizePropertyName': False}},
                    'name': 'product_Name',
                    'tokenization': 'word'
                },
                {
                    'dataType': ['number'],
                    'description': 'Price of the product.',
                    'indexFilterable': True,
                    'indexSearchable': False,
                    'moduleConfig': {'text2vec-openai': {'skip': False, 'vectorizePropertyName': False}},
                    'name': 'price'
                },
                {
                    'dataType': ['number'],
                    'description': 'Rating of the product.',
                    'indexFilterable': True,
                    'indexSearchable': False,
                    'moduleConfig': {'text2vec-openai': {'skip': False, 'vectorizePropertyName': False}},
                    'name': 'rating'
                },
                {
                    'dataType': ['text'],
                    'description': 'Description of the product.',
                    'indexFilterable': True,
                    'indexSearchable': True,
                    'moduleConfig': {'text2vec-openai': {'skip': False, 'vectorizePropertyName': False}},
                    'name': 'description',
                    'tokenization': 'word'
                },
                {
                    'dataType': ['text'],
                    'description': 'Feature of the product.',
                    'indexFilterable': True,
                    'indexSearchable': True,
                    'moduleConfig': {'text2vec-openai': {'skip': False, 'vectorizePropertyName': False}},
                    'name': 'features',
                    'tokenization': 'word'
                }
            ],
            'replicationConfig': {'factor': 1},
            'shardingConfig': {
                'virtualPerPhysical': 128,
                'desiredCount': 1,
                'actualCount': 1,
                'desiredVirtualCount': 128,
                'actualVirtualCount': 128,
                'key': '_id',
                'strategy': 'hash',
                'function': 'murmur3'
            },
            'vectorIndexConfig': {
                'skip': False,
                'cleanupIntervalSeconds': 300,
                'maxConnections': 64,
                'efConstruction': 128,
                'ef': -1,
                'dynamicEfMin': 100,
                'dynamicEfMax': 500,
                'dynamicEfFactor': 8,
                'vectorCacheMaxObjects': 1000000000000,
                'flatSearchCutoff': 40000,
                'distance': 'cosine',
                'pq': {
                    'enabled': False,
                    'bitCompression': False,
                    'segments': 0,
                    'centroids': 256,
                    'trainingLimit': 100000,
                    'encoder': {'type': 'kmeans', 'distribution': 'log-normal'}
                }
            },
            'vectorIndexTy

In [16]:
1024 / 2

512.0

## Automerging Index and Store in Vectore DB

In [65]:
product_index = build_automerging_index(
    documents = docs,
    llm = llm,
    embed_model=embed_model,
    save_dir="merging_index",
    chunk_sizes = [2048, 1024, 512],
    # prefix = "Product_auto_merge_index"
)
# sentence_index = build_sentence_window_index(
#     document = document,
#     llm = llm,
#     embed_model=embed_model,
#     save_dir="sentence_index"
# )



## Build Weaviate Index

In [18]:
# vector_store = WeaviateVectorStore(weaviate_client=client, class_prefix="Product_index")
# storage_context = StorageContext.from_defaults(vector_store=vector_store)


# product_index = VectorStoreIndex.from_documents([document], storage_context=storage_context, 
#                 service_context = service_context)



# 2. RETRIEVAL

## Build SQL Query Engine

In [66]:
# set up text2SQL prompt
sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["product_table"],
)

## Build Vector Query Engine

In [67]:
vector_query_engine  = get_automerging_query_engine(
                    automerging_index = product_index,
                     similarity_top_k=12,
                    rerank_top_n=2,)

# vector_query_engine = product_index.as_query_engine()

## Tell llama index what tool to use

In [68]:
# from llama_index.tools.query_engine import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine = sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over a product listing table containing: "
        "Price , Rating, Description, and Features of each product"
    ),
)
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for answering semantic questions about product listing",
)

## Create a router


In [69]:
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=([sql_tool] + [vector_tool]),
)

## Queries


In [70]:
response = query_engine.query("What is the Rating for the product whose name is Portable Solar Charger?")
# print(str(response))
display_response(response)

**`Final Response:`** The rating for the product named Portable Solar Charger is 4.0.

In [24]:
response = query_engine.query("Tell me about Wireless Bluetooth Earbuds products in the product listing ")
display_response(response)

**`Final Response:`** The Ultimate Wireless Bluetooth Earbuds are a top choice for audiophiles who prioritize premium sound quality and convenience. With advanced Bluetooth 5.0 technology, these earbuds offer seamless connectivity and crystal-clear audio. They provide high-fidelity sound with deep bass and clear treble, and also feature noise-canceling technology for an immersive listening experience. With 20 hours of playtime on a single charge, you can enjoy your music for extended periods. The ergonomic design ensures a secure and comfortable fit, and the touch controls make it easy to control playback and calls. Additionally, these earbuds have an IPX7 waterproof rating, making them suitable for outdoor activities.

In [25]:
rprint(response)

Response(
    response='The Ultimate Wireless Bluetooth Earbuds are a top choice for audiophiles looking for premium sound 
quality and convenience. With advanced Bluetooth 5.0 technology, these earbuds offer seamless connectivity and 
crystal-clear audio. They feature high-fidelity sound with deep bass and clear treble, as well as noise-canceling 
technology for an immersive listening experience. With 20 hours of playtime on a single charge, these earbuds are 
perfect for long listening sessions. The ergonomic design ensures a secure and comfortable fit, and the touch 
controls make playback and calls easy. Additionally, these earbuds have an IPX7 waterproof rating, making them 
suitable for outdoor activities.',
    source_nodes=[
        NodeWithScore(
            node=TextNode(
                id_='80dc47a4-e56d-4340-9a34-ac4330dbec6f',
                embedding=None,
                metadata={},
                excluded_embed_metadata_keys=[],
                excluded_llm_metadata_keys=[],
                relationships={},
                text='[(\'Ultimate Wireless Bluetooth Earbuds\', 79.99, 4.4, "Elevate your music experience with 
our Ultimate Wireless Bluetooth Earbuds. These earbuds are designed for audiophiles who demand premium sound 
quality and convenience. With advanced Bluetooth 5.0 technology, you\'ll enjoy seamless connectivity and 
crystal-clear audio.", \'- High-fidelity sound with deep bass and clear treble. - Noise-canceling technology for an
immersive listening experience. - 20 hours of playtime on a single charge. - Ergonomic design for a secure and 
comfortable fit. - Touch controls for easy playback and calls. - IPX7 waterproof rating for...\')]',
                start_char_idx=None,
                end_char_idx=None,
                text_template='{metadata_str}\n\n{content}',
                metadata_template='{key}: {value}',
                metadata_seperator='\n'
            ),
            score=None
        )
    ],
    metadata={
        '80dc47a4-e56d-4340-9a34-ac4330dbec6f': {},
        'sql_query': "SELECT Product_Name, Price, Rating, Description, Features\nFROM product_table\nWHERE 
Product_Name LIKE '%Wireless Bluetooth Earbuds%'",
        'result': [
            (
                'Ultimate Wireless Bluetooth Earbuds',
                79.99,
                4.4,
                "Elevate your music experience with our Ultimate Wireless Bluetooth Earbuds. These earbuds are 
designed for audiophiles who demand premium sound quality and convenience. With advanced Bluetooth 5.0 technology, 
you'll enjoy seamless connectivity and crystal-clear audio.",
                '- High-fidelity sound with deep bass and clear treble. - Noise-canceling technology for an 
immersive listening experience. - 20 hours of playtime on a single charge. - Ergonomic design for a secure and 
comfortable fit. - Touch controls for easy playback and calls. - IPX7 waterproof rating for...'
            )
        ],
        'col_keys': ['Product_Name', 'Price', 'Rating', 'Description', 'Features'],
        'selector_result': MultiSelection(
            selections=[
                SingleSelection(
                    index=0,
                    reason='Choice 1 is the most relevant because it mentions translating a natural language query 
into a SQL query over a product listing table, which would be useful for retrieving information about Wireless 
Bluetooth Earbuds products in the product listing.'
                )
            ]
        )
    }
)

In [26]:
response = query_engine.query("How many product cost 79.99 in the product listing?")
display_response(response)

**`Final Response:`** There are 2 products in the product listing that cost 79.99.

In [27]:
rprint(response)

Response(
    response='There are 2 products in the product listing that cost 79.99.',
    source_nodes=[
        NodeWithScore(
            node=TextNode(
                id_='613c821c-0ed8-4f28-81c8-9fc5531b920d',
                embedding=None,
                metadata={},
                excluded_embed_metadata_keys=[],
                excluded_llm_metadata_keys=[],
                relationships={},
                text='[(2,)]',
                start_char_idx=None,
                end_char_idx=None,
                text_template='{metadata_str}\n\n{content}',
                metadata_template='{key}: {value}',
                metadata_seperator='\n'
            ),
            score=None
        )
    ],
    metadata={
        '613c821c-0ed8-4f28-81c8-9fc5531b920d': {},
        'sql_query': 'SELECT COUNT(*) FROM product_table WHERE Price = 79.99',
        'result': [(2,)],
        'col_keys': ['COUNT(*)'],
        'selector_result': MultiSelection(
            selections=[
                SingleSelection(
                    index=0,
                    reason='Choice 1 is the most relevant because it mentions translating a natural language query 
into a SQL query over a product listing table, which would be necessary to find the number of products that cost 
79.99 in the product listing.'
                )
            ]
        )
    }
)

In [71]:
response = query_engine.query("How many product are similar to Wireless Bluetooth Earbuds products?")
display_response(response)

> Merging 1 nodes into parent node.
> Parent node id: bf395249-9811-44fc-b758-e6b54cf05f71.
> Parent node text: Description: Elevate your music experience with our Ultimate Wireless Bluetooth Earbuds. These ea...

> Merging 1 nodes into parent node.
> Parent node id: 41a2b0da-3087-49ad-bf13-4fb60d8a7339.
> Parent node text: Description: Say goodbye to tangled cables with our Wireless Charging Pad. Simply place your devi...

> Merging 1 nodes into parent node.
> Parent node id: c4289c7f-2158-48cf-9cb6-cbd1a1134ec5.
> Parent node text: Description: Achieve your fitness goals with our Fitness Tracker Smartwatch. Monitor your activit...

> Merging 1 nodes into parent node.
> Parent node id: 32298bc7-6587-4a0c-a782-42e6086573b9.
> Parent node text: Description: Elevate your style with our Luxury Leather Wallet. Handcrafted from premium leather,...

> Merging 1 nodes into parent node.
> Parent node id: a575913a-23ac-495e-a9d2-8a8e871c4b7c.
> Parent node text: Description: Immerse yourself in

**`Final Response:`** There are two products that are similar to Wireless Bluetooth Earbuds products.

In [ ]:
rprint(response)

In [72]:
response = query_engine.query("list the products whose descriptions are similar.")
display_response(response)

> Merging 1 nodes into parent node.
> Parent node id: 258e6bee-6460-4615-977e-36ee94589de9.
> Parent node text: Description: Savor the soothing flavors of our Organic Herbal Tea Collection. Crafted from handpi...

> Merging 1 nodes into parent node.
> Parent node id: cbb15697-a3fa-48b9-b2cd-bcf5cad50adb.
> Parent node text: Description: Experience ultimate comfort with our Organic Bamboo Bed Sheets. These luxuriously so...

> Merging 1 nodes into parent node.
> Parent node id: bf395249-9811-44fc-b758-e6b54cf05f71.
> Parent node text: Description: Elevate your music experience with our Ultimate Wireless Bluetooth Earbuds. These ea...

> Merging 1 nodes into parent node.
> Parent node id: 5f07a55b-3911-40e3-a07a-c171ffeeb945.
> Parent node text: Description: Brew barista-quality coffee at home with our Professional Espresso Machine. This mac...

> Merging 1 nodes into parent node.
> Parent node id: 4a78319c-8c49-49ab-9713-440d769a86bc.
> Parent node text: Description: Get a full-body wor

**`Final Response:`** The products whose descriptions are similar are the Organic Herbal Tea Collection and the Portable Solar Charger.

In [47]:
print(response.metadata)

{
    '6a6ebaef-32cc-4b08-9df7-27cdd34978f3': {},
    'selector_result': MultiSelection(
        selections=[
            SingleSelection(
                index=1,
                reason='Choice 2 is more relevant because it specifically mentions answering semantic questions 
about product listing, which includes determining similarity between product descriptions.'
            )
        ]
    )
}

In [31]:
rprint(data.loc[data.Product_Name.isin(('Organic Bamboo Bed Sheets', 'Virtual Reality Headset')), "Description"].values)

['Experience ultimate comfort with our Organic Bamboo Bed Sheets. These luxuriously soft sheets are made from 
sustainable and breathable bamboo fibers.'
 'Immerse yourself in virtual worlds with our Virtual Reality Headset. Experience gaming, entertainment, and 
exploration like never before.']

In [73]:
response = query_engine.query("Which product has 'heart rate' in its description")
display_response(response)

**`Final Response:`** The product that has 'heart rate' in its description is the Fitness Tracker Smartwatch.

In [84]:
rprint(response)

Response(
    response="The product that has 'heart rate' in its description is the Fitness Tracker Smartwatch.",
    source_nodes=[
        NodeWithScore(
            node=TextNode(
                id_='0dbf45a2-3a3c-497b-bb2e-d57eca866a37',
                embedding=None,
                metadata={},
                excluded_embed_metadata_keys=[],
                excluded_llm_metadata_keys=[],
                relationships={},
                text="[('Fitness Tracker Smartwatch',)]",
                start_char_idx=None,
                end_char_idx=None,
                text_template='{metadata_str}\n\n{content}',
                metadata_template='{key}: {value}',
                metadata_seperator='\n'
            ),
            score=None
        )
    ],
    metadata={
        '0dbf45a2-3a3c-497b-bb2e-d57eca866a37': {},
        'sql_query': "SELECT Product_Name\nFROM product_table\nWHERE Description LIKE '%heart rate%'",
        'result': [('Fitness Tracker Smartwatch',)],
        'col_keys': ['Product_Name'],
        'selector_result': MultiSelection(
            selections=[
                SingleSelection(
                    index=0,
                    reason='Choice 1 is most relevant because it mentions translating a natural language query into
a SQL query over a table containing product listings, which would include descriptions of each product. This 
suggests that it would be able to handle a query about the description of a product.'
                )
            ]
        )
    }
)

In [58]:
response = query_engine.query("What is the sum of all prices for all products whose price is $79.99?")
display_response(response)

**`Final Response:`** The sum of all prices for all products whose price is $79.99 is $159.98.

In [87]:
rprint(response.metadata)

{
    'f419eef1-48cd-4084-a103-47d7dd9f3bc0': {},
    'sql_query': 'SELECT SUM(Price) FROM product_table WHERE Price = 79.99;',
    'result': [(159.98,)],
    'col_keys': ['SUM(Price)'],
    'selector_result': MultiSelection(
        selections=[
            SingleSelection(
                index=0,
                reason='Choice 1 is the most relevant because it mentions translating a natural language query into
a SQL query over a table containing product listings, including the price. This suggests that it can be used to 
find the sum of all prices for all products.'
            )
        ]
    )
}

## Self Correcting query

### Retry Query Engine

In [74]:
query_response_evaluator = RelevancyEvaluator()
retry_query_engine = RetryQueryEngine(
    query_engine, query_response_evaluator
)
retry_response = retry_query_engine.query("What is the sum of all prices for all products whose price is $79.99?")
display_response(retry_response)

**`Final Response:`** The sum of all prices for all products whose price is $79.99 is $159.98.

### Retry Source Query Engine

In [75]:
retry_source_query_engine = RetrySourceQueryEngine(
    query_engine, query_response_evaluator
)
retry_source_response = retry_source_query_engine.query("list the products whose descriptions are similar.")
display_response(retry_source_response)


> Merging 1 nodes into parent node.
> Parent node id: 258e6bee-6460-4615-977e-36ee94589de9.
> Parent node text: Description: Savor the soothing flavors of our Organic Herbal Tea Collection. Crafted from handpi...

> Merging 1 nodes into parent node.
> Parent node id: cbb15697-a3fa-48b9-b2cd-bcf5cad50adb.
> Parent node text: Description: Experience ultimate comfort with our Organic Bamboo Bed Sheets. These luxuriously so...

> Merging 1 nodes into parent node.
> Parent node id: bf395249-9811-44fc-b758-e6b54cf05f71.
> Parent node text: Description: Elevate your music experience with our Ultimate Wireless Bluetooth Earbuds. These ea...

> Merging 1 nodes into parent node.
> Parent node id: 5f07a55b-3911-40e3-a07a-c171ffeeb945.
> Parent node text: Description: Brew barista-quality coffee at home with our Professional Espresso Machine. This mac...

> Merging 1 nodes into parent node.
> Parent node id: 4a78319c-8c49-49ab-9713-440d769a86bc.
> Parent node text: Description: Get a full-body wor

**`Final Response:`** The products whose descriptions are similar are the Organic Herbal Tea Collection and the Portable Solar Charger.

## Customize Retry Guideline Query Engine

In [76]:
# Guideline eval
guideline_eval = GuidelineEvaluator(
    guidelines=DEFAULT_GUIDELINES
    + "\nThe response should not be overly long.\n"
    "The response should try to summarize where possible.\n"
    "First, answer the question\n"
    "Second provide the reason, why you choose that answer.\n"
)  # just for example

In [77]:
query = "list the products whose descriptions are similar."
typed_response = (
    retry_source_response if isinstance(retry_source_response, Response) else retry_source_response.get_response()
)
eval = guideline_eval.evaluate_response(query, typed_response)
print(f"Guideline eval evaluation result: {eval.feedback}")

feedback_query_transform = FeedbackQueryTransformation(resynthesize_query=True)
transformed_query = feedback_query_transform.run(query, {"evaluation": eval})
print(f"Transformed query: {transformed_query.query_str}")

Guideline eval evaluation result: The response does not fully answer the query. It only lists two products without 
providing any explanation or reasoning for why they are considered similar. Additionally, the response does not 
provide any statistics or numbers to support the claim. It is also vague and lacks specificity.

Transformed query: Here is a previous bad answer.
The products whose descriptions are similar are the Organic Herbal Tea Collection and the Portable Solar Charger.
Here is some feedback from the evaluator about the response given.
The response does not fully answer the query. It only lists two products without providing any explanation or 
reasoning for why they are considered similar. Additionally, the response does not provide any statistics or 
numbers to support the claim. It is also vague and lacks specificity.
Now answer the question.
Please return the modified query below.

In [78]:
retry_guideline_query_engine = RetryGuidelineQueryEngine(
    query_engine, guideline_eval, resynthesize_query=True, 
     max_retries=3,)
retry_guideline_response = retry_guideline_query_engine.query(query)
# rprint(retry_guideline_response.response)
display_response(retry_guideline_response)

> Merging 1 nodes into parent node.
> Parent node id: 258e6bee-6460-4615-977e-36ee94589de9.
> Parent node text: Description: Savor the soothing flavors of our Organic Herbal Tea Collection. Crafted from handpi...

> Merging 1 nodes into parent node.
> Parent node id: cbb15697-a3fa-48b9-b2cd-bcf5cad50adb.
> Parent node text: Description: Experience ultimate comfort with our Organic Bamboo Bed Sheets. These luxuriously so...

> Merging 1 nodes into parent node.
> Parent node id: bf395249-9811-44fc-b758-e6b54cf05f71.
> Parent node text: Description: Elevate your music experience with our Ultimate Wireless Bluetooth Earbuds. These ea...

> Merging 1 nodes into parent node.
> Parent node id: 5f07a55b-3911-40e3-a07a-c171ffeeb945.
> Parent node text: Description: Brew barista-quality coffee at home with our Professional Espresso Machine. This mac...

> Merging 1 nodes into parent node.
> Parent node id: 4a78319c-8c49-49ab-9713-440d769a86bc.
> Parent node text: Description: Get a full-body wor

**`Final Response:`** I apologize for the confusion earlier. Unfortunately, I was unable to find any additional products with similar descriptions in the database. However, I can provide more information about the two products mentioned earlier, the Ultimate Wireless Bluetooth Earbuds and the SmartHome Security Camera System, if you would like.

## Another example of self correcting query

In [79]:
# test another query

query = "What is the sum of all prices for all products whose price is $79.99?"
response = query_engine.query(query)
display_response(response)
# print(str(response))
typed_response = (
    response if isinstance(response, Response) else response.get_response()
)
eval = guideline_eval.evaluate_response(query, typed_response)
print(f"Guideline eval evaluation result: {eval.feedback}")

feedback_query_transform = FeedbackQueryTransformation(resynthesize_query=True)
transformed_query = feedback_query_transform.run(query, {"evaluation": eval})
print(f"Transformed query: {transformed_query.query_str}")

**`Final Response:`** The sum of all prices for all products whose price is $79.99 is $159.98.

Guideline eval evaluation result: The response does not fully answer the query. It only provides the sum of prices 
for products with a price of $79.99, but it does not specify the reason for choosing that answer. Additionally, the
response is not specific and does not use statistics or numbers. It should provide more information and be more 
concise.

Transformed query: Here is a previous bad answer.
The sum of all prices for all products whose price is $79.99 is $159.98.
Here is some feedback from the evaluator about the response given.
The response does not fully answer the query. It only provides the sum of prices for products with a price of 
$79.99, but it does not specify the reason for choosing that answer. Additionally, the response is not specific and
does not use statistics or numbers. It should provide more information and be more concise.
Now answer the question.
What is the total cost of all products priced at $79.99?

In [80]:
retry_guideline_query_engine = RetryGuidelineQueryEngine(
    query_engine, guideline_eval, resynthesize_query=True,
    max_retries=2,
)
retry_guideline_response = retry_guideline_query_engine.query(query)
display_response(retry_guideline_response)

**`Final Response:`** The total sum of prices for all products with a price of $79.99 is $159.98. This answer is based on the SQL query that calculates the sum of prices for products with a price of $79.99.